## pysenic的输入本质是counts矩阵。我们可以改变细胞数，也可以改变基因数（1w高变基因）。
## 如果输入的 counts 太大，计算时间太长，用5w细胞，1w基因是合适的。

In [1]:
import os
import sys
import anndata as ad
import pandas as pd
import numpy as np
import scanpy as sc
import argparse
import loompy as lp;

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-j_lqsplr because the default path (/home/stereonote/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
adata = ad.read('/data/work/02.result/others/ST_NC/02.result/05.scenic/anno_0818.h5ad')
adata

AnnData object with n_obs × n_vars = 240713 × 46068
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'sample', 'celltype_0818', 'Time_category'
    var: 'Gene'

In [3]:
# 2) 准备原始计数层：优先用已有 counts，其次用 raw.X，再不行才用 X
if 'counts' in adata.layers:
    adata.layers['counts'] = adata.layers['counts']
elif adata.raw is not None and adata.raw.X is not None:
    adata.layers['counts'] = adata.raw.X
else:
    adata.layers['counts'] = adata.X  # 警告：若非整数计数，仅作为折中用；PySCENIC严格应使用原始计数


In [5]:
# 3) 去掉全零基因（显著降维提速）
#   注意：filter_genes 会在 adata.var 里做子集，counts层会同步切
sc.pp.filter_genes(adata, min_counts=1)

In [8]:
# 4) 计算 HVG（按 sample 分批可减批次效应；没有该列会自动忽略）

# !pip install --user scikit-misc

batch_key = 'sample' if 'sample' in adata.obs.columns else None
sc.pp.highly_variable_genes(
    adata,
    layer='counts',          # 指定用原始计数层
    flavor='seurat_v3',      # 避开 seurat 老算法的分箱/溢出问题
    n_top_genes = 10_000,
    batch_key=batch_key,
    check_values=False       # 跳过可能触发溢出的检查
)

hvg_mask = adata.var['highly_variable'].values
hvg_n = int(hvg_mask.sum())
print(f'HVG 选出 {hvg_n} 个基因（目标 {N_HVG}）。')

HVG 选出 10000 个基因（目标 10000）。


In [10]:
# 5) 仅选择 HVG 基因；细胞不变
adata_hvg = adata[:, hvg_mask].copy()
# 6) 确保写入 loom 的是“计数矩阵”
adata_hvg.X = adata_hvg.layers['counts']  # 让 X 指向原始计数（稀疏也可）
adata_hvg

AnnData object with n_obs × n_vars = 240713 × 10000
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'sample', 'celltype_0818', 'Time_category'
    var: 'Gene', 'n_counts', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'highly_variable_nbatches'
    uns: 'hvg'
    layers: 'counts'

In [11]:
# 7) 随机抽 50,000 细胞
N_CELLS = 50_000
n_total = adata_hvg.n_obs
np.random.seed(42)  # 固定随机种子，保证可重复
if n_total > N_CELLS:
    keep_idx = np.random.choice(n_total, N_CELLS, replace=False)
    adata_sub = adata_hvg[keep_idx, :].copy()
    
print(f"抽样后：{adata_sub.n_obs} cells × {adata_sub.n_vars} genes")
adata_sub


抽样后：50000 cells × 10000 genes


AnnData object with n_obs × n_vars = 50000 × 10000
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'sample', 'celltype_0818', 'Time_category'
    var: 'Gene', 'n_counts', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'highly_variable_nbatches'
    uns: 'hvg'
    layers: 'counts'

In [13]:
out_loom = '/data/work/02.result/others/ST_NC/02.result/05.scenic/anno_0818_5w_1wHVG.loom'

# 8) 为 loom 补充常用属性（Gene/CellID），减少无关负担
adata_sub.var['Gene'] = adata_sub.var_names
adata_sub.obs['CellID'] = adata_sub.obs_names

# 可选：只保留少量必要的 obs/var 列，减小文件尺寸
keep_obs = [c for c in ['CellID','orig.ident','sample','celltype_0818','Time_category'] if c in adata_sub.obs.columns]
keep_var = [c for c in ['Gene'] if c in adata_sub.var.columns]
adata_sub.obs = adata_sub.obs[keep_obs]
adata_sub.var = adata_sub.var[keep_var]

# 8) 写 loom（AnnData 自带写法，能正确处理稀疏，且省内存）
adata_sub.write_loom(out_loom, write_obsm_varm=False)
print(f'已写出 loom：{out_loom}')

已写出 loom：/data/work/02.result/others/ST_NC/02.result/05.scenic/anno_0818_50kHVG.loom


In [ ]:
import numpy as np
import anndata as ad
import scanpy as sc

# 输入/输出文件路径
in_h5ad = '/data/work/02.result/others/ST_NC/02.result/05.scenic/anno_0818.h5ad'
out_loom = '/data/work/02.result/others/ST_NC/02.result/05.scenic/anno_0818_50kHVG.loom'

N_HVG = 10_000
N_CELLS = 50_000

# 1) 读取数据
adata = ad.read(in_h5ad)
print(f"原始数据: {adata.n_obs} cells × {adata.n_vars} genes")

# 2) 准备 counts 层（PySCENIC 需要原始计数）
if 'counts' in adata.layers:
    adata.layers['counts'] = adata.layers['counts']
elif adata.raw is not None and adata.raw.X is not None:
    adata.layers['counts'] = adata.raw.X
else:
    adata.layers['counts'] = adata.X  # 如果不是整数计数，只能先用 X 代替

# 3) 去掉全零基因
sc.pp.filter_genes(adata, min_counts=1)
print(f"去零基因后: {adata.n_obs} cells × {adata.n_vars} genes")

# 4) 计算 HVG
batch_key = 'sample' if 'sample' in adata.obs.columns else None
sc.pp.highly_variable_genes(
    adata,
    layer='counts',
    flavor='seurat_v3',
    n_top_genes=N_HVG,
    batch_key=batch_key,
    check_values=False
)

hvg_mask = adata.var['highly_variable'].values
print(f"HVG 基因数: {hvg_mask.sum()}")

# 5) 只保留 HVG 基因
adata = adata[:, hvg_mask].copy()
adata.X = adata.layers['counts']   # 确保 loom 写 counts 矩阵
print(f"保留 HVG 后: {adata.n_obs} cells × {adata.n_vars} genes")

# 6) 随机抽样 50k 细胞
np.random.seed(42)
if adata.n_obs > N_CELLS:
    keep_idx = np.random.choice(adata.n_obs, N_CELLS, replace=False)
    adata = adata[keep_idx, :].copy()
print(f"抽样后: {adata.n_obs} cells × {adata.n_vars} genes")

# 7) 精简 obs/var，保留必要信息
adata.var['Gene'] = adata.var_names
adata.obs['CellID'] = adata.obs_names

keep_obs = [c for c in ['CellID','orig.ident','sample','celltype_0818','Time_category'] if c in adata.obs.columns]
adata.obs = adata.obs[keep_obs]
adata.var = adata.var[['Gene']]

# 8) 写 loom 文件
adata.write_loom(out_loom, write_obsm_varm=False)
print(f"已写出 loom 文件: {out_loom}")
